In [1]:
import numpy as np
import import_ipynb
from env_v2 import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


importing Jupyter notebook from env_v2.ipynb
pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env=environement()
info={}
env.env_init(info)

In [3]:
class a2c_v2:
    def __init__(self, info={}):
        self.n_actions = info.get("n_actions",4) 
        self.input_shape=info.get("input_shape",(10,10,1)) 
        self.EPISODES = info.get("episodes",100000) 
        self.lr = info.get("lr",0.001) 
        self.gamma=info.get("gamma",0.99)
        self.Actor, self.Critic = self.create_model(input_shape=self.input_shape, n_actions=self.n_actions, lr=self.lr)
        self.states, self.actions, self.rewards = [], [], []
        
    def create_model(self,input_shape,n_actions,lr):
        inputs= keras.Input(shape=input_shape)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(inputs)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(x)
        block_1_output = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
        x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
        block_2_output = layers.add([x, block_1_output])
        x=layers.Flatten()(block_2_output)
        action = layers.Dense(n_actions, activation="softmax")(x)
        value = layers.Dense(1)(x)

        Actor = keras.Model(inputs = inputs, outputs = action)
        Actor.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=lr))

        Critic = keras.Model(inputs = inputs, outputs = value)
        Critic.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=lr))

        return Actor, Critic
    def remember(self, state, action, reward):
        self.states.append(np.reshape(state,(1,10,10,1)))
        action_onehot = np.zeros([self.n_actions])
        action_onehot[action] = 1
        self.actions.append(action_onehot)
        self.rewards.append(reward)
    def act(self, state):
        prediction = self.Actor.predict(np.reshape(state,(1,10,10,1)))[0]
        action = np.random.choice(self.n_actions, p=prediction)
        return action
    def discount_rewards(self, reward):
        gamma=self.gamma    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add
        return discounted_r
    def replay(self):
        states = np.vstack(self.states)
        actions = np.vstack(self.actions)
        discounted_r = self.discount_rewards(self.rewards)
        values = self.Critic.predict(states)[:, 0]
        advantages = discounted_r - values
        # training Actor and Critic networks
        self.Actor.fit(states, actions, sample_weight=advantages, epochs=1, verbose=0)
        self.Critic.fit(states, discounted_r, epochs=1, verbose=0)
        self.states, self.actions, self.rewards = [], [], []
    def step(self, action):
        reward, next_state, done,score = env.step(action)
        return next_state, reward, done,score
    def run(self):
        for e in range(self.EPISODES):
            state = env.start()
            done = False
            while not done:
                action = self.act(state)
                next_state, reward, done,score= self.step(action)
                self.remember(state, action, reward)
                state = next_state
                if done:
                    print("Episode :",e," Score :",score)
                    self.replay()
    def save(self):
        self.Actor.save('tmp/Actor')
        self.Critic.save(self.Model_name + 'tmp/Critic')
    def load(self, Actor_name, Critic_name):
        self.Actor = load_model('tmp/Actor')
        self.Critic = load_model('tmp/Critic')



In [4]:
a=a2c_v2()

In [ ]:
a.run()

Episode : 0  Score : 0
Episode : 1  Score : 0
Episode : 2  Score : 0
Episode : 3  Score : 0
Episode : 4  Score : 0
Episode : 5  Score : 0
Episode : 6  Score : 0
Episode : 7  Score : 0
Episode : 8  Score : 0
Episode : 9  Score : 0
Episode : 10  Score : 0
Episode : 11  Score : 0
Episode : 12  Score : 0
Episode : 13  Score : 2
Episode : 14  Score : 0
Episode : 15  Score : 0
Episode : 16  Score : 1
Episode : 17  Score : 2
Episode : 18  Score : 0
Episode : 19  Score : 0
Episode : 20  Score : 0
Episode : 21  Score : 0
Episode : 22  Score : 0
Episode : 23  Score : 0
Episode : 24  Score : 0
Episode : 25  Score : 0
Episode : 26  Score : 0
Episode : 27  Score : 0
Episode : 28  Score : 0
Episode : 29  Score : 0
Episode : 30  Score : 3
Episode : 31  Score : 1
Episode : 32  Score : 0
Episode : 33  Score : 0
Episode : 34  Score : 0
Episode : 35  Score : 0
Episode : 36  Score : 1
Episode : 37  Score : 0
Episode : 38  Score : 0
Episode : 39  Score : 0
Episode : 40  Score : 0
Episode : 41  Score : 0
Ep